### Import thư viện

In [37]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('weather.csv')

In [3]:
df["date"] = pd.to_datetime(df["date"])

### Phân tích và chọn mô hình phù hợp

Thêm cột `avg_temp` vào data bằng cách lấy trung bình của cột `max` (nhiệt độ có giá trị cao nhất) và `min` (nhiệt độ có giá trị nhỏ nhất)

In [4]:
df["avg_temp"] = (df["max"] + df["min"]) / 2
df.head()

,province,max,min,wind,wind_d,rain,humidi,cloud,pressure,date,avg_temp
0,Bac Lieu,27,22,17,NNE,6.9,90,71,1010,2009-01-01,24.5
1,Bac Lieu,31,25,20,ENE,0.0,64,24,1010,2010-01-01,28.0
2,Bac Lieu,29,24,14,E,0.0,75,45,1008,2011-01-01,26.5
3,Bac Lieu,30,24,30,E,0.0,79,52,1012,2012-01-01,27.0
4,Bac Lieu,31,25,20,ENE,0.0,70,24,1010,2013-01-01,28.0


Tính nhiệt độ trung bình của các tỉnh thành có trong dữ liệu

In [26]:
df_grouped = df.groupby(["date"]).agg({"avg_temp": "mean"}).reset_index()

df_grouped.head(10)

,date,avg_temp
0,2009-01-01,20.3625
1,2009-01-02,20.3250
2,2009-01-03,19.7375
3,2009-01-04,21.4125
4,2009-01-05,22.4000
5,2009-01-06,23.2625
6,2009-01-07,21.4625
7,2009-01-08,20.5500
8,2009-01-09,19.9375
9,2009-01-10,18.5750


Vẽ biểu đồ thể hiện nhiệt độ trung bình của khu vực xuất hiện trong dữ liệu

In [31]:
fig = px.line(df_grouped, x="date", y="avg_temp", title='Average Temperature in Data', height=500)

fig.show()

Từ biểu đồ trên thấy được mỗi năm nhiệt độ có đương giống như một vòng cung (rounding top). Tổng thể, biểu đồ biểu diễn giống như những con sóng.

Nhìn vào biểu đồ mẫu của mô hình Prophet, thấy được một sự tương quan nhất định với biểu đồ trên khi đều xuất hiện các sóng (rounding top) nên nhóm em sẽ chọn mô hình này để dự đoán nhiệt độ

![image](picture/prophet.png)

Giới thiệu về Prophet: mô hình dự đoán cho dữ liệu time series của Facebook. Mô hình phù hợp với dữ liệu thiếu, thay đổi theo xu hướng, các trường hợp ngoại lệ

### Mô hình hóa và dự đoán

Sử dụng mô hình Facebook Prophet để dự đoán, mỗi `province` làm một model riêng

Lấy 80% dữ liệu làm tập train, 10% làm tập validate, 10% làm tập test

In [38]:
def predict(df: pd.DataFrame, province: str):

    location_df = df[df["province"] == province]

    location_df = location_df.reset_index()[["date", "avg_temp"]].rename(
        {"date": "ds", "avg_temp": "y"}, axis=1
    )

    location_df.sort_values("ds", inplace=True)
    location_df.reset_index(drop=True, inplace=True)
    
    train_df = location_df.iloc[: int(location_df.shape[0] * 0.8)]
    test_df = location_df.iloc[int(location_df.shape[0] * 0.8) :]

    location_model = Prophet()

    location_model.fit(train_df)

    future = location_model.make_future_dataframe(periods=test_df.shape[0])

    forecast = location_model.predict(future)

    return forecast, train_df, test_df, location_model

Hàm đánh giá mô hình thông qua evaluate metrics: MSE, MAE, MAPE

In [22]:
def model_evaluation(test_df: pd.DataFrame, forecast: pd.DataFrame):

    mse = mean_squared_error(test_df["y"], forecast["yhat"][-test_df.shape[0] :])
    mae = mean_absolute_error(test_df["y"], forecast["yhat"][-test_df.shape[0] :])
    mape = mean_absolute_percentage_error(test_df["y"], forecast["yhat"][-test_df.shape[0] :])

    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")

Ha Noi, Ho Chi Minh, Can Tho, Nha Trang được chọn để thử nghiệm mô hình

In [39]:
hanoi_forecast, hanoi_train, hanoi_test, hanoi_model = predict(df, "Ha Noi")
hcm_forecast, hcm_train, hcm_test, hcm_model = predict(df, "Ho Chi Minh City")
ct_forecast, ct_train, ct_test, ct_model = predict(df, "Can Tho")
nt_forecast, nt_train, nt_test, nt_model = predict(df, "Nha Trang")

16:29:24 - cmdstanpy - INFO - Chain [1] start processing
16:29:25 - cmdstanpy - INFO - Chain [1] done processing
16:29:26 - cmdstanpy - INFO - Chain [1] start processing
16:29:27 - cmdstanpy - INFO - Chain [1] done processing
16:29:28 - cmdstanpy - INFO - Chain [1] start processing
16:29:29 - cmdstanpy - INFO - Chain [1] done processing
16:29:30 - cmdstanpy - INFO - Chain [1] start processing
16:29:31 - cmdstanpy - INFO - Chain [1] done processing


In [34]:
print("Ha Noi")
model_evaluation(hanoi_test, hanoi_forecast)
print()

print("Ho Chi Minh City")
model_evaluation(hcm_test, hcm_forecast)
print()

print("Can Tho")
model_evaluation(ct_test, ct_forecast)
print()

print("Nha Trang")
model_evaluation(nt_test, nt_forecast)

Ha Noi
MSE: 8.542339704320648
MAE: 2.307878937061575
MAPE: 0.10031419990280206

Ho Chi Minh City
MSE: 2.41931466259282
MAE: 1.1030008728413796
MAPE: 0.039364493809862566

Can Tho
MSE: 2.037915660804406
MAE: 1.0186173229716564
MAPE: 0.037192359516059044

Nha Trang
MSE: 3.640827186686138
MAE: 1.584760209828971
MAPE: 0.06374800106921687


Vẽ biểu đồ thể hiện đường dự đoán nhiệt độ của Hà Nội

In [36]:
fig = plot_plotly(hanoi_model, hanoi_forecast)
fig.show()

In [48]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hanoi_train['ds'],
    y=hanoi_train['y'],
    mode='lines',
    name='Train data',
    line=dict(color='green')
))

fig.add_trace(go.Scatter(
    x=hanoi_test['ds'],
    y=hanoi_test['y'],
    mode='lines',
    name='Test data',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=hanoi_forecast["ds"][-hanoi_test.shape[0]:],
    y=hanoi_forecast["yhat"][-hanoi_test.shape[0]:],
    mode='lines',
    name='Forecast data',
    line=dict(color='blue')
))

fig.update_layout(
    title='Temperature Forecast',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Temperature'),
)

fig.show()

In [49]:
df.nunique()

province      40
max           43
min           31
wind          51
wind_d        16
rain        1298
humidi        77
cloud        101
pressure      49
date        4549
avg_temp      67
dtype: int64

### Đề xuất giải pháp

Do muốn đảm bảo độ chính xác ổn nên nhóm em thực hiện cách tạo một mô hình cho từng giá trị `province`, data chỉ có 39 tỉnh nên việc tạo 39 trained model cũng tốn khoảng thời gian không nhiều. Nhưng khi mở rộng phạm vi địa lý lên 64 tỉnh thành hoặc ra các nước khác sẽ xuất hiện nhiều nhược điểm với giải pháp này: tốn thời gian, cần hệ thống khác để quản lý các mô hình...

Giải pháp thay thế: 

- Tạo hoặc sử dụng mô hình tiên tiến hơn
- Sử dụng các kiểu file dành riêng cho thời thiết: EPW (EnergyPlus Weather Format), CLM (ESP-r weather format), WEA (Daysim weather format)
